In [18]:
import pandas as pd
import dash
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objs as go
from dash import html
from dash import dcc
import dash_core_components as core_components

# Load your data
file_path = '/Users/myPath/Name1.csv'
dataTableNetherlandsAPI = pd.read_csv(file_path)

# Initialize Dash app
app = dash.Dash(__name__)

# Additional data preparation steps
dataTableNetherlandsAPI['Date'] = pd.to_datetime(dataTableNetherlandsAPI['Date'])
dataTableNetherlandsAPI['Year'] = dataTableNetherlandsAPI['Date'].dt.year
dataTableNetherlandsAPI['Month'] = dataTableNetherlandsAPI['Date'].dt.month
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dataTableNetherlandsAPI['DayOfWeek'] = pd.Categorical(dataTableNetherlandsAPI['Date'].dt.day_name(), categories=day_order, ordered=True)

available_cities = dataTableNetherlandsAPI['City'].unique()
available_years = sorted(dataTableNetherlandsAPI['Year'].unique(), reverse=True)

# Define app layout
app.layout = html.Div([
    html.H1("API Levels in Netherlands"),
    
    html.P("Welcome to the Interactive dashboard on Netherlands API levels. "
           "Air Pollution Index (API) provides information about the air quality in different cities. "
           "Explore the visualizations below to analyze API levels from 2018 to 2023."),

    html.H1("API in Netherlands cities by Year"),

    html.Label("Select City and Year"),
    dcc.Dropdown(
        id='city-dropdown',
        options=[{'label': city, 'value': city} for city in available_cities],
        value=available_cities[0]
    ),
    dcc.Dropdown(
        id='year-dropdown',
        options=[{'label': year, 'value': year} for year in available_years],
        value=available_years[0]
    ),

    dcc.Graph(id='bar-chart'),

    html.H1("Monthly Mean API by City and Year"),

    html.Label("Select Cities and Year to Compare"),
    dcc.Dropdown(
        id='city-dropdown2',
        options=[{'label': city, 'value': city} for city in available_cities],
        value=available_cities[0]
    ),
    dcc.Dropdown(
        id='city-dropdown-compare',
        options=[{'label': city, 'value': city} for city in available_cities],
        value=available_cities[1]
    ),
    dcc.Dropdown(
        id='year-dropdown2',
        options=[{'label': year, 'value': year} for year in available_years],
        value=available_years[0]
    ),

    dcc.Graph(id='line-chart'),

    html.H1("Monthly Mean API Levels by Day of the Week"),

    html.Label("Select City"),
    dcc.Dropdown(
        id='city-dropdown3',
        options=[{'label': city, 'value': city} for city in available_cities],
        value=available_cities[0]
    ),

    dcc.Graph(id='heatmap-chart'),

    html.H1("API Time Series"),

    html.Label("Select City and Year for Time Series"),
    dcc.Dropdown(
        id='city-dropdown-calplot',
        options=[{'label': city, 'value': city} for city in available_cities],
        value=available_cities[0]
    ),
    dcc.Dropdown(
        id='year-dropdown-calplot',
        options=[{'label': year, 'value': year} for year in available_years],
        value=available_years[0]
    ),

    dcc.Graph(id='calplot-chart'),

])

# Define callbacks to update plots based on user input
@app.callback(
    Output('bar-chart', 'figure'),
    [Input('city-dropdown', 'value'),
     Input('year-dropdown', 'value')]
)
def update_bar_chart(selected_city, selected_year):
    # Filter data based on selected city and year
    filtered_data = dataTableNetherlandsAPI[(dataTableNetherlandsAPI['City'] == selected_city) & (dataTableNetherlandsAPI['Year'] == selected_year)]

    # Calculate the maximum API levels for each month
    monthly_max = filtered_data.groupby('Month')['API'].max().reset_index()

    # Create bar chart
    fig = px.bar(monthly_max, x='Month', y='API', labels={'API': 'Maximum API Level'}, title=f'Maximum API Levels in {selected_city} by Month')
    
    return fig

@app.callback(
    Output('line-chart', 'figure'),
    [Input('city-dropdown2', 'value'),
     Input('city-dropdown-compare', 'value'),
     Input('year-dropdown2', 'value')]
)
def update_line_chart(city1, city2, year2):
    # Filter data for the first city
    filtered_data1 = dataTableNetherlandsAPI[(dataTableNetherlandsAPI['City'] == city1) & (dataTableNetherlandsAPI['Year'] == year2)]
    # Filter data for the second city
    filtered_data2 = dataTableNetherlandsAPI[(dataTableNetherlandsAPI['City'] == city2) & (dataTableNetherlandsAPI['Year'] == year2)]

    # Calculate monthly mean API for each city
    monthly_stats1 = filtered_data1.groupby(['Year', 'Month', 'City'])['API'].mean().reset_index()
    monthly_stats2 = filtered_data2.groupby(['Year', 'Month', 'City'])['API'].mean().reset_index()

    # Add a color column to each dataframe
    monthly_stats1['Color'] = 'blue'  # Assign a color for the first city (e.g., blue)
    monthly_stats2['Color'] = 'red'   # Assign a fixed color for the second city (e.g., red)

    # Combine dataframes
    combined_data = pd.concat([monthly_stats1, monthly_stats2])

    # Add a YearMonth column
    combined_data['Year, Month'] = combined_data[['Year', 'Month']].apply(lambda x: f'{x[0]}-{x[1]:02}', axis=1)

    # Create line chart with labels
    fig = go.Figure()

    for city, color in zip([city1, city2], ['blue', 'red']):
        city_data = combined_data[combined_data['City'] == city]
        fig.add_trace(go.Scatter(
            x=city_data['Year, Month'],
            y=city_data['API'],
            mode='lines+markers',
            name=city,
            line=dict(color=color),
            text=city_data['API'].round(2).astype(str),  # Convert to string to ensure hover text is displayed
            hoverinfo='text+y',
        ))

    fig.update_layout(
        title='Monthly Mean API Levels by City and Year',
        xaxis=dict(title='Year, Month'),
        yaxis=dict(title='Mean API'),
        showlegend=True,
    )

    return fig

@app.callback(
    Output('heatmap-chart', 'figure'),
    [Input('city-dropdown3', 'value')]
)
def update_heatmap_chart(selected_city):
    filtered_data = dataTableNetherlandsAPI[dataTableNetherlandsAPI['City'] == selected_city]
    monthly_stats = filtered_data.groupby(['Month', 'DayOfWeek'])['API'].mean().reset_index()
    pivot_table = monthly_stats.pivot(index='Month', columns='DayOfWeek', values='API')

    days = list(day_order)
    months = list(range(1, 13))

    trace = go.Heatmap(z=pivot_table.values,
                      x=days,
                      y=months,
                      colorscale='Viridis',
                      colorbar=dict(title='Mean API'))

    layout = go.Layout(title=f'Monthly Mean API Levels in {selected_city} by Day of the Week',
                       xaxis=dict(title='Day of the Week'),
                       yaxis=dict(title='Month'))

    fig = go.Figure(data=[trace], layout=layout)

    return fig


@app.callback(
    Output('calplot-chart', 'figure'),
    [Input('city-dropdown-calplot', 'value'),
     Input('year-dropdown-calplot', 'value')]
)
def update_calplot_chart(selected_city, selected_year):
    # Filter data for the selected city and year
    filtered_data = dataTableNetherlandsAPI.loc[(dataTableNetherlandsAPI['City'] == selected_city) & (dataTableNetherlandsAPI['Year'] == selected_year), :].copy()

    # Convert 'Date' column to datetime format
    filtered_data['Date'] = pd.to_datetime(filtered_data['Date'])

    # Create a calendar heatmap using Plotly with a red-blue color scale
    fig = go.Figure(go.Heatmap(
        z=filtered_data['API'],
        x=filtered_data['Date'].dt.day,
        y=filtered_data['Date'].dt.month,
        colorscale='RdBu',
        reversescale=True,
        colorbar=dict(title='API Levels'),
    ))

    fig.update_layout(
        title=f'API Time Series Calendar Heatmap in {selected_city} for the Year {selected_year}',
        xaxis_title='Day',
        yaxis_title='Month',
    )

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, host='your host code', port=XXXX)